In [267]:
import numpy as np
import pandas as pd
from datetime import datetime as dt, timedelta as tmd
from warnings import filterwarnings
filterwarnings('ignore')

from xAPIConnector.xAPIConnector import *
from xAPIConnector.config import user_id, pwd

In [268]:
period_dict = {
    '1min': 1,
    '5min': 5,
    '15min': 15,
    '30min': 30,
    '1h': 60,
    '4h': 240,
    '1D': 1440,
    '1W': 10080,
    '1M': 43200
}

def XTB_to_pandas(response):
    data = pd.DataFrame.from_dict(response['returnData']['rateInfos'])
    digits = response['returnData']['digits']

    data['Date'] = data['ctm'].apply(lambda x: dt.fromtimestamp(x/1000))
    data['Price'] = (data['open'] + data['close'])/(10**digits)
    data = data.loc[:, ['Date', 'Price']]
    data = data.set_index('Date')
    data = data.iloc[:, 0]

    return data

class DataLoader:
    def __init__(self, user_id, pwd):
        
        self.user_id = user_id
        self.pwd = pwd
        
        self.client = None
        
    def connect(self, verbose: bool = True):    
        self.client = APIClient()
        if verbose: print(f"[{dt.now()}] Loguję do API...")
        self.client.execute(loginCommand(self.user_id, self.pwd))
    
    def disconnect(self, verbose: bool = True):
        if verbose: print(f"[{dt.now()}] Wylogowuję z API...")
        self.client.disconnect()
        
    def getData(self,
                symbols: list[str],
                start_date: str,
                end_date: str | None = None,
                interval: str = '1min',
                verbose: bool = True):
        
        self.connect(verbose)
        
        finalData = {}
        
        end_date = (dt.strptime(end_date, '%Y-%m-%d %H:%M:%S') if end_date is not None else dt.now())
        endUNIXTIME = int(dt.timestamp(end_date) * 1000)
        start_date = dt.strptime(start_date, '%Y-%m-%d %H:%M:%S') + tmd(days=-1)
        startUNIXTIME = int(dt.timestamp(start_date) * 1000)
        
        for symbol in symbols:
            args = {'info': {
                    'end': endUNIXTIME,
                    'start': startUNIXTIME,
                    'symbol': symbol,
                    'period': period_dict[interval]
            }}
            if verbose: print(f"\tWysyłam zapytanie do API...", end=' ')
            response = self.client.commandExecute('getChartRangeRequest', arguments=args)
            print(response)
            finalData[symbol] = XTB_to_pandas(response)
        
        self.disconnect(verbose)
        
        return pd.DataFrame(finalData)

In [269]:
symbols = ['BITCOIN', 'ETHEREUM']
start = "2025-01-08 00:00:00"
end = "2025-01-09 00:00:00"

In [270]:
dl = DataLoader(user_id, pwd)
data = dl.getData(symbols=symbols, start_date=start, end_date=end)
data

[2025-01-13 11:51:41.823770] Loguję do API...
	Wysyłam zapytanie do API... {'status': False, 'errorCode': 'BE103', 'errorDescr': 'Not logged - getChartRangeRequest'}


KeyError: 'returnData'

# Klasyfikacja

Klasyfikatory można budować na mnóstwo różnych sposobów. Ogólnie trzeba się zdecydować, na ilu obserwacjach wstecz ma się opierać klasyfikacja. Liczbę tych obserwacji nazwiemy 'window' i przyjmiemy jako parametr.

Na razie ustawiamy klasyfikację na podstawie poprzednich 4 godzin.

Pierwszy pomysł to klasyfikacja na podstawie stóp zwrotu, klasy również będziemy budować na podstawie stóp zwrotu. Metoda budowania klas zostanie omówiona dalej.

In [188]:
interval = '5m' # dane 5-minutowe
returns = prices.pct_change().dropna()

window = int(4*60/int(interval[:-1]))

In [198]:
def generate_X_y(data: pd.DataFrame | pd.Series, window: int, skip: int) -> tuple:   
    # można ustawić okna zachodzące (0 < skip < window)
    # można ustawić niezachodzące (skip >= window)
    assert skip > 0, "Pętla w kodzie nigdy się nie zakończy..."
    
    # Generujemy 'okna'
    X = pd.DataFrame(columns=range(window-1))
    y = pd.Series()
    
    i = len(data)
    count = 0
    while i >= window:
        temp_y = data.iloc[i-window:i]
        
        X.loc[count, :] = temp_y.iloc[:-1].values
        y.loc[count] = temp_y.iloc[-1]
        
        i = i - skip    
        count += 1
        
    return np.array(X), np.array(y)

### Regresja logistyczna

In [237]:
from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import LeaveOneOut

Pierwszy pomysł na zdefiniowanie klas to przyjęcie '1' jeśli stopa zwrotu jest $>threshold$ oraz '0' w p.p.

Próg odcięcia można podać jako parametr i już po pierwszych testach widać, że dla prostej klasyfikacji typu "czy zwrot > 0" skuteczność jest około $50\%$. Im wyższy próg, tym lepsza skuteczność.

In [244]:
skip = window
X, y = generate_X_y(data=returns, window=window, skip=skip)

threshold = 0.002
y = (np.array(y) > threshold)*1.0

clf = LogisticRegression().fit(X, y)
print(f"Skuteczność treningowa modelu: {clf.score(X, y):.4%}")

Skuteczność treningowa modelu: 87.0968%


Teraz podzielimy dane na treningowe i testowe, po czym sprawdzimy jakość predykcji

In [245]:
train_test_ratio = 0.7
len_train = int(train_test_ratio*X.shape[0])

Xtrain = X[:len_train, :]
ytrain = y[:len_train]

Xtest = X[len_train:, :]
ytest = y[len_train:]

print(f"[INFO] W zbiorze treningowym znaduje się {Xtrain.shape[0]} obserwacji.")
print(f"[INFO] W zbiorze testowym znajduje się {Xtest.shape[0]} obserwacji.")

[INFO] W zbiorze treningowym znaduje się 21 obserwacji.
[INFO] W zbiorze testowym znajduje się 10 obserwacji.


In [246]:
clf = LogisticRegression().fit(Xtrain, ytrain)
print(f"Skuteczność treningowa modelu: {clf.score(Xtrain, ytrain):.4%}")
print(f"Skuteczność testowa modelu: {clf.score(Xtest, ytest):.4%}")

Skuteczność treningowa modelu: 90.4762%
Skuteczność testowa modelu: 80.0000%
